<a href="https://colab.research.google.com/github/nabilaaaa24/data_mining/blob/main/Praktikum_Modul_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kegiatan Modul 6**

Content Based Filtering : Anime

Tulislan code sesuai instruksi pada sel yang telah disediakan

## Downlaod dan extract dataset dari situs [kaggle.com](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020)

In [ ]:
! pip install kaggle

In [ ]:
! cp /content/drive/MyDrive/Dataset/kaggle.json ~/.kaggle/kaggle.json

cp: cannot stat '/content/drive/MyDrive/Dataset/kaggle.json': No such file or directory


In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets download -d 'hernan4444/anime-recommendation-database-2020'

Dataset URL: https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020
License(s): CC0-1.0
 96% 637M/661M [00:02<00:00, 284MB/s]
100% 661M/661M [00:02<00:00, 273MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

In [ ]:
dataset_zip = zipfile.ZipFile('anime-recommendation-database-2020.zip', 'r')

dataset_zip.extractall()

dataset_zip.close()

## Import library dan load dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Untuk tugas ini cukup gunakan data "anime_with_synopsis.csv"
# Tampilkan 5 data teratas
df = pd.read_csv('/content/anime_with_synopsis.csv')
df.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


## Tangani data yang kosong (missing value)

In [ ]:
########################
# Preprocessing
########################
df.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64

In [ ]:
df['sypnopsis'].fillna('', inplace=True)
df

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


Hasil penanganan missing value

In [ ]:
df.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    0
dtype: int64

## Selanjutnya lakukan feature extraction pada fitur "Genres" menggunakan TF-IDF

In [ ]:
########################
# Creating TF-IDF Matrix
########################
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Genres'])
tfidf_matrix.shape

(16214, 46)

## Menghitung cosine similarity dari TF-IDF diatas

In [ ]:
########################
# Creating Cosine Similarity Matrix
########################
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

cosine_sim.shape

(16214, 16214)

## Membuat Content Based Recomender

In [ ]:
########################
# Creating Content Based Recommender
########################
indices = pd.Series(df.index, index=df['Name']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    anime_indices = [i[0] for i in sim_scores]
    return df[['Name', 'Genres']].iloc[anime_indices]

## Mencoba Content Based Recomender

Mempilkan 5 Judul Anime dengan yang memiliki nilai kemiripan tertinggi berdasarkan genre dari kata kunci judul anime yang dimasukkan

In [ ]:
# hasil rekomendasi anime dengan keyword "Paprika"
get_recommendations('Paprika')

,Name,Genres
824,Higurashi no Naku Koro ni,"Mystery, Dementia, Horror, Psychological, Supe..."
15171,Higurashi no Naku Koro ni Gou,"Mystery, Dementia, Horror, Psychological, Supe..."
16211,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe..."
13265,Yakusoku no Neverland,"Sci-Fi, Mystery, Horror, Psychological, Thrill..."
6598,Higurashi no Naku Koro ni Kaku: Outbreak,"Mystery, Horror, Psychological, Thriller"


In [ ]:
# hasil rekomendasi anime dengan keyword "Monster"
get_recommendations('Monster')

,Name,Genres
6598,Higurashi no Naku Koro ni Kaku: Outbreak,"Mystery, Horror, Psychological, Thriller"
2495,Death Note: Rewrite,"Mystery, Police, Psychological, Supernatural, ..."
10449,B: The Beginning,"Action, Mystery, Police, Psychological, Supern..."
13404,B: The Beginning Succession,"Action, Mystery, Police, Psychological, Supern..."
289,Mousou Dairinin,"Mystery, Dementia, Police, Psychological, Supe..."


In [ ]:
# hasil rekomendasi anime dengan keyword "Naruto"
get_recommendations('Naruto')

,Name,Genres
1508,Naruto: Shippuuden,"Action, Adventure, Comedy, Super Power, Martia..."
11640,Boruto: Jump Festa 2016 Special,"Action, Adventure, Comedy, Super Power, Martia..."
206,Rekka no Honoo,"Action, Adventure, Martial Arts, Shounen, Supe..."
5518,Naruto: Honoo no Chuunin Shiken! Naruto vs. Ko...,"Action, Adventure, Martial Arts, Shounen, Supe..."
6158,Naruto: Shippuuden Movie 6 - Road to Ninja,"Action, Adventure, Super Power, Martial Arts, ..."
